In [1]:
!pip install -U bitsandbytes


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.1/76.1 MB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 27.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 24.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 29.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 8.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 8.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 57.5 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling 

In [5]:
!pip install -U numpy;
!pip install -U transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.2/10.2 MB 94.1 MB/s eta 0:00:00
  Attempting uninstall: transformers
    Found existing installation: transformers 4.49.0
    Uninstalling transformers-4.49.0:
      Successfully uninstalled transformers-4.49.0


In [1]:
import transformers
from transformers import (AutoModelForCausalLM,
                          BitsAndBytesConfig,
                          AutoTokenizer,
                          TrainingArguments
                          )
import torch

In [2]:
#parameters
quant_config = BitsAndBytesConfig(use_4bit=True,
                                  bnb_4bit_compute_dtype=torch.bfloat16,
                                  bnb_4bit_quant_type='nf4')
model_name = "NousResearch/Llama-2-7b-hf"


## Load pretrained

In [ ]:
model = AutoModelForCausalLM.from_pretrained(model_name,
                                             quantization_config=quant_config,
                                             device_map='auto')

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)


#### Training arguments

## Data


In [5]:
!git clone https://github.com/wmt-conference/wmt21-news-systems.git

fatal: destination path 'wmt21-news-systems' already exists and is not an empty directory.


In [ ]:
src_path = "/content/wmt21-news-systems/txt/references/newstest2021.de-en.ref.A.en"
ref_path = "/content/wmt21-news-systems/txt/sources/newstest2021.de-en.src.de"


In [ ]:
source_str = open(src_path).read()
ref_str = open(ref_path).read()
source_str

In [8]:
PROMPT_TEMPLATE = "{source}: {source_sentence} {dest}: "


In [9]:
!pip install datasets

from datasets import Dataset
def build_parallel_ds(source_str,ref_str):
  ref_list = ref_str.split("\n")
  src_list = source_str.split("\n")
  assert len(ref_list) == len(src_list)
  ds = {"src":[],"ref":[],"prompt":[]}
  for i in range(len(ref_list)):
    ds["src"].append(src_list[i])
    ds["ref"].append(ref_list[i])
    prompt = PROMPT_TEMPLATE.format(
        source="English",
        dest="German",
        source_sentence=src_list[i])
    ds["prompt"].append(prompt)

  return Dataset.from_dict(ds)

In [10]:
!pip install unbabel-comet

from comet import download_model, load_from_checkpoint

comet_path = download_model("Unbabel/wmt20-comet-da")
comet_model = load_from_checkpoint(comet_path)



Fetching 5 files:   0%|          | 0/5 [00:00<?, ?it/s]

INFO:pytorch_lightning.utilities.migration.utils:Lightning automatically upgraded your loaded checkpoint from v1.3.5 to v2.5.1. To apply the upgrade to your files permanently, run `python -m pytorch_lightning.utilities.upgrade_checkpoint ../root/.cache/huggingface/hub/models--Unbabel--wmt20-comet-da/snapshots/87819f4d6d4f17e0d1752cc9e0ccfa2064997219/checkpoints/model.ckpt`
/usr/local/lib/python3.11/dist-packages/pytorch_lightning/core/saving.py:195: Found keys that are not in the model state dict but in the checkpoint: ['encoder.model.embeddings.position_ids']


## Zero-shot : COMET evaluationn

In [49]:



def eval_comet_zeroshot(parallel_ds, model, tokenizer, comet_model, prompt=PROMPT_TEMPLATE):


  scores = []
  def translate(batch):
    prompts = batch["prompt"]
    input(f"{prompts}")
    input_ids = tokenizer(prompts)[0] # Return PyTorch tensors
    ref_toks = tokenizer(batch["ref"])[0]
    input(f"{ref_toks}")

    max_len = torch.max(ref_toks.shape[0])
    len_prompt = input_ids.shape[0]
    input_ids = torch.tensor([input_ids], device=model.device)

    with torch.no_grad():
      output_tokens = model.generate(input_ids,
                                      max_new_tokens=max_len,
                                      num_beams=2,
                                      early_stopping=True,
                                      batched=True)
    a = input("après gen, continuer ?")

    output = tokenizer.decode(output_tokens[0][len_prompt:])
    return {"mt" : output}

  a = input("continuer ?")

  parallel_ds = parallel_ds.map(translate, batched=True,batch_size=4,writer_batch_size=4)

  with torch.no_grad():
    scores.append(comet_model.predict(parallel_ds,batch_size=16,batched= True))
  return scores


In [ ]:
ds = build_parallel_ds(source_str, ref_str)
print(ds)
param_size = 0
for param in model.parameters():
    param_size += param.nelement() * param.element_size()
buffer_size = 0
for buffer in model.buffers():
    buffer_size += buffer.nelement() * buffer.element_size()

size_all_mb = (param_size + buffer_size) / 1024**2
print('model size: {:.3f}MB'.format(size_all_mb))

scores = eval_comet_zeroshot(ds.take(2), model, tokenizer, comet_model)

print(ds.take(2))
# print(scores)
# print(np.mean(scores))


Dataset({
    features: ['src', 'ref', 'prompt'],
    num_rows: 1001
})
model size: 3588.508MB
continuer ?t


In [38]:
import gc
gc.collect()

302

In [ ]:
print(list(map(lambda x: x[0],scores)))
#COMET 20

In [ ]:
print(list(map(lambda x: x[0],scores)))
#beam search=5

In [ ]:
print(list(map(lambda x: x[0],scores)))
#beam search=2